## Step 1. 加载模型与Tokenizer

In [1]:
import os
import sys

from transformers import AutoTokenizer

sys.path.append(os.path.abspath('../..'))
from sfl.model.gpt2.gpt2_split import GPT2SplitLMHeadModel

cache_dir = '/root/autodl-tmp/sfl/models'  # 模型的缓存位置，需要修改
save_dir = '/root/autodl-tmp/sfl/models/checkpoints'
tokenizer = AutoTokenizer.from_pretrained("gpt2", cache_dir=cache_dir)
model = GPT2SplitLMHeadModel.from_pretrained("gpt2", cache_dir=cache_dir)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = 50256

In [2]:
from sfl.utils import calculate_rouge
# 恢复的评价指标选用ROUGE

def evaluate(epc, md, attacker, tok, test_data_loader):
    md.eval()
    attacker.eval()
    dl_len = len(test_data_loader)
    with torch.no_grad():
        rouge_1, rouge_2, rouge_l_f1, rouge_l_p, rouge_l_r = 0, 0, 0, 0, 0
        for step, batch in tqdm(enumerate(test_data_loader), total=dl_len):
            input_ids = batch['input_ids'].to(md.device)
            attention_mask = batch['input_att_mask'].to(md.device)
            inter = md(input_ids=input_ids, attention_mask=attention_mask)
            logits = attacker(inter)
            result = calculate_rouge(tok, logits, batch['input_text'])
            rouge_1 += result['rouge-1']['f']
            rouge_2 += result['rouge-2']['f']
            rouge_l_f1 += result['rouge-l']['f']
            rouge_l_p += result['rouge-l']['p']
            rouge_l_r += result['rouge-l']['r']
    print(
        f'Epoch {epc} Test Rouge_1: {rouge_1 / dl_len}, Rouge_2: {rouge_2 / dl_len}, Rouge_l_f1: {rouge_l_f1 / dl_len}, Rouge_l_p: {rouge_l_p / dl_len}, Rouge_l_r: {rouge_l_r / dl_len}')
    path = save_dir + '/attacker/'
    os.makedirs(path, exist_ok=True)
    torch.save(attacker.state_dict(), path+f'epoch_{epc}_rouge_{rouge_l_f1 / dl_len}.pt')
    md.train(True)
    attacker.train(True)
    return rouge_1 / dl_len, rouge_2 / dl_len, rouge_l_f1 / dl_len, rouge_l_p / dl_len, rouge_l_r / dl_len

### 加载数据集

In [3]:
from datasets import load_dataset
from torch.utils.data import DataLoader

def encode(examples):
    # same input and output
    input = tokenizer(examples["goal"], padding="max_length")
    return {'input_ids': input['input_ids'], 'input_att_mask': input['attention_mask'],
            'output_ids': input['input_ids'], 'output_att_mask': input['attention_mask'],
            "input_text": examples["goal"], "output_text": examples["sol1"]}

dataset = load_dataset('piqa')['train']
dataset_test = load_dataset('piqa')['validation']
dataset = dataset.map(encode)
dataset_test = dataset_test.map(encode)
dataset.set_format(type="torch", columns=["input_ids", "input_att_mask", "output_ids", "output_att_mask", "input_text"])
dataset_test.set_format(type="torch",
                        columns=["input_ids", "input_att_mask", "output_ids", "output_att_mask", "input_text"])
dataloader = DataLoader(dataset, batch_size=6)
dataloader_test = DataLoader(dataset_test, batch_size=6)


### 切分模型

In [4]:
from sfl.utils import FLConfig
model.config_sfl(FLConfig(collect_intermediates=False,
                          split_point_1=2, # 第0～1层为top，余下的都是trunk
                          split_point_2=999,
                          attack_mode='b2tr' # 攻击的输出是bottom-to-trunk中间输出
                          ),
                 param_keeper=None)
# model = model.convert_to_lora_model(restore_top_bottom=False)
model.print_split_model()

=================Split-gpt2=================
==================Top Layers==================
==================Trunk Layers==================

transformer.h.2:[ln_1.weight: (768,), ln_1.bias: (768,), attn.c_attn.weight: (768, 2304), attn.c_attn.bias: (2304,), attn.c_proj.weight: (768, 768), attn.c_proj.bias: (768,), ln_2.weight: (768,), ln_2.bias: (768,), mlp.c_fc.weight: (768, 3072), mlp.c_fc.bias: (3072,), mlp.c_proj.weight: (3072, 768), mlp.c_proj.bias: (768,)]

transformer.h.3:[ln_1.weight: (768,), ln_1.bias: (768,), attn.c_attn.weight: (768, 2304), attn.c_attn.bias: (2304,), attn.c_proj.weight: (768, 768), attn.c_proj.bias: (768,), ln_2.weight: (768,), ln_2.bias: (768,), mlp.c_fc.weight: (768, 3072), mlp.c_fc.bias: (3072,), mlp.c_proj.weight: (3072, 768), mlp.c_proj.bias: (768,)]

transformer.h.4:[ln_1.weight: (768,), ln_1.bias: (768,), attn.c_attn.weight: (768, 2304), attn.c_attn.bias: (2304,), attn.c_proj.weight: (768, 768), attn.c_proj.bias: (768,), ln_2.weight: (768,), ln_2.bia

### 训练Attack Model

In [5]:
from torch.optim import Adam
from sfl.model.attack_model import GPT2AttackModel
from sfl.utils import get_best_gpu, calc_unshift_loss
from tqdm.notebook import tqdm
import torch


def get_output(text, encoder_model, attack_model):
    t = tokenizer(text, return_tensors="pt", add_special_tokens=False)
    inter = encoder_model(t['input_ids'].to(device), attention_mask=t['attention_mask'].to(device))
    res = attack_model(inter)
    r = tokenizer.decode(res.argmax(dim=-1)[-1], skip_special_tokens=True)
    return r


# 开始训练Attack Model
device = get_best_gpu()
attack_model = GPT2AttackModel(model.config)
optimizer = Adam(attack_model.parameters(), lr=1e-3)
model.to(device)
attack_model.to(device)
epoch = 5
evaluate(0,model,attack_model,tokenizer,dataloader_test)
with tqdm(total=epoch * len(dataloader)) as pbar:
    for epc in range(epoch):
        model.train(True)
        rouge_1, rouge_2, rouge_l_f1, rouge_l_p, rouge_l_r = 0, 0, 0, 0, 0
        for step, batch in enumerate(dataloader):
            optimizer.zero_grad()
            input_ids, labels = batch['input_ids'].to(device), batch['output_ids'].to(device)
            attention_mask = batch['input_att_mask'].to(device)
            intermediate = model(input_ids=input_ids, labels=labels, attention_mask=attention_mask)
            logits = attack_model(intermediate)
            loss = calc_unshift_loss(logits, labels)
            loss.backward()
            optimizer.step()
            # 计算训练的ROGUE
            res = calculate_rouge(tokenizer, logits, batch['input_text'])
            rouge_1 += res['rouge-1']['f']
            rouge_2 += res['rouge-2']['f']
            rouge_l_f1 += res['rouge-l']['f']
            rouge_l_p += res['rouge-l']['p']
            rouge_l_r += res['rouge-l']['r']
            pbar.set_description(f'Epoch {epc} Loss {loss.item():.5f}, Rouge_1 {rouge_1 / (step + 1):.4f}')
            if step % 300 == 0:
                q = "To mix food coloring with sugar, you can"
                print(q, "==>", get_output(q, model, attack_model))
            pbar.update(1)
        rouge_1 /= len(dataloader)
        rouge_2 /= len(dataloader)
        rouge_l_f1 /= len(dataloader)
        rouge_l_p /= len(dataloader)
        rouge_l_r /= len(dataloader)
        print(
            f'Epoch {epc} Train Rouge_1: {rouge_1}, Rouge_2: {rouge_2}, Rouge_l_f1: {rouge_l_f1}, Rouge_l_p: {rouge_l_p}, Rouge_l_r: {rouge_l_r}')
        # 计算测试集上的ROGUE
        evaluate(epc, model, attack_model, tokenizer, dataloader_test)

  0%|          | 0/307 [00:00<?, ?it/s]

Epoch 0 Test Rouge_1: 5.4288814223670027e-05, Rouge_2: 0.0, Rouge_l_f1: 5.4288814223670027e-05, Rouge_l_p: 3.877772607414301e-05, Rouge_l_r: 9.048136083966703e-05


  0%|          | 0/13430 [00:00<?, ?it/s]

To mix food coloring with sugar, you can ==> TF arguing Manager Italianademprogress ClownEP existed
To mix food coloring with sugar, you can ==> To,, you can
To mix food coloring with sugar, you can ==> To with, you can
To mix food coloring with sugar, you can ==> To mix food toy with cake, you can
To mix food coloring with sugar, you can ==> To mix food with egg, you can
To mix food coloring with sugar, you can ==> To mix food? with sugar, you can
To mix food coloring with sugar, you can ==> To mix food with sugar, you can
To mix food coloring with sugar, you can ==> To mix food with sugar, you can
To mix food coloring with sugar, you can ==> To mix food white with sugar, you can
Epoch 0 Train Rouge_1: 0.5446797326462616, Rouge_2: 0.3712868895887097, Rouge_l_f1: 0.5443499999457075, Rouge_l_p: 0.5989855911314934, Rouge_l_r: 0.51519047757485


  0%|          | 0/307 [00:00<?, ?it/s]

...

Epoch 2 Test Rouge_1: 0.857998595559974, Rouge_2: 0.773012674545034, Rouge_l_f1: 0.857998595559974, Rouge_l_p: 0.866011417660453, Rouge_l_r: 0.8528574216745725
To mix food coloring with sugar, you can ==> To mix food coloring with sugar, you can
To mix food coloring with sugar, you can ==> To mix food coloring with sugar, you can
To mix food coloring with sugar, you can ==> To mix food coloring with sugar, you can
To mix food coloring with sugar, you can ==> To mix food coloring with sugar, you can
To mix food coloring with sugar, you can ==> To mix food coloring with sugar, you can
To mix food coloring with sugar, you can ==> To mix food coloring with sugar, you can
To mix food coloring with sugar, you can ==> To mix food coloring with sugar, you can
To mix food coloring with sugar, you can ==> To mix food coloring with sugar, you can
To mix food coloring with sugar, you can ==> To mix food coloring with sugar, you can
Epoch 3 Train Rouge_1: 0.890173322867279, Rouge_2: 0.8262424690612

  0%|          | 0/307 [00:00<?, ?it/s]

Epoch 3 Test Rouge_1: 0.8923710748086362, Rouge_2: 0.8102987189473285, Rouge_l_f1: 0.8923710748086362, Rouge_l_p: 0.8945170249930939, Rouge_l_r: 0.8909958360949178
To mix food coloring with sugar, you can ==> To mix food coloring with sugar, you can
To mix food coloring with sugar, you can ==> To mix food coloring with sugar, you can
To mix food coloring with sugar, you can ==> To mix food coloring with sugar, you can
To mix food coloring with sugar, you can ==> To mix food coloring with sugar, you can
To mix food coloring with sugar, you can ==> To mix food coloring with sugar, you can
To mix food coloring with sugar, you can ==> To mix food coloring with sugar, you can
To mix food coloring with sugar, you can ==> To mix food coloring with sugar, you can
To mix food coloring with sugar, you can ==> To mix food coloring with sugar, you can
To mix food coloring with sugar, you can ==> To mix food coloring with sugar, you can
Epoch 4 Train Rouge_1: 0.9218208405568472, Rouge_2: 0.86219896

  0%|          | 0/307 [00:00<?, ?it/s]

Epoch 4 Test Rouge_1: 0.9131500286422674, Rouge_2: 0.8325680055026835, Rouge_l_f1: 0.9131500286422674, Rouge_l_p: 0.9137924955155265, Rouge_l_r: 0.912888086430041
